In [1]:
import plistlib
import mutagen
from pathlib import Path
from urllib.parse import unquote, urlparse
import subprocess
import os

In [2]:
os.chdir("/Users/toph/Local/Projects/vibechecker-scratch")

## Define classes

In [3]:
class Vibe():
    def __init__(self, vibe_dict, con):
        self.vibe_id = vibe_dict["vibe_id"]
        self.name = vibe_dict["name"]
        self.playlist_id = vibe_dict["playlist_id"]
        self.parent_id = vibe_dict["parent_id"]
        self.type = vibe_dict["type"]

    def __repr__(self):
        return self.name

In [4]:
vibe = []

In [5]:
class Track():
    def __init__(self, track_dict, con):
        self.track_id = track_dict["track_id"]
        self.name = track_dict["name"]
        self.artist = track_dict["artist"]
        self.album = track_dict["album"]
        self.tracknum = track_dict["tracknum"]
        self.con = con

    def __repr__(self):
        return f"{self.artist} — {self.name}"

    def vibes(self):
        track_vibes = con.execute("SELECT * FROM track_vibes WHERE track_id=?", (self.track_id,))
        vibe_ids = [v["vibe_id"] for v in track_vibes]
        sql = "SELECT * FROM vibes WHERE vibe_id IN ({seq}) AND type != 'folder'".format(seq=",".join(["?"]*len(vibe_ids))) # Wow this is obnoxious
        vibes = [Vibe(v, self.con) for v in con.execute(sql, vibe_ids)]
        return vibes
    
    def add_vibe(self, vibe):
        vibe = {
            "track_vibe_id": int(str(self.track_id) + str(vibe.vibe_id)),
            "vibe_id": vibe.vibe_id,
            "track_id": self.track_id,
            "value": 1.0
        }
        columns = ', '.join(vibe.keys())
        placeholders = ', '.join('?' * len(vibe))
        print(vibe.values())
        print(type(vibe.values()))
        print([type(v) for v in vibe.values()])
        sql = """
        INSERT OR IGNORE INTO track_vibes
            ({})
        VALUES
            ({})""".format(
                columns, placeholders
            )
        print(sql)
        self.con.cursor().execute(sql, tuple(vibe.values()))
        self.con.commit()

## Load and query tables

In [6]:
import sqlite3

In [7]:
con = sqlite3.connect("vibechecker.db")

In [8]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d
con.row_factory = dict_factory

In [9]:
cursor = con.cursor()

In [10]:
command = """
SELECT * FROM tracks
"""
tracks = []
for track_dict in cursor.execute(command):
    track = Track(track_dict, con)
    if len(track.vibes()) is not 0:
        tracks.append(track)

In [11]:
command = """
SELECT * FROM vibes
"""
vibes = []
for vibe_dict in cursor.execute(command):
    vibe = Vibe(vibe_dict, con)
    if vibe.type != "folder":
        vibes.append(vibe)

In [12]:
t = tracks[7]

In [13]:
t

Superpoze — Shelter

In [14]:
t.vibes()

[Energy 2, Longing / Melancholy, Endings]

In [15]:
v = [v for v in vibes if v.name == "Pretty"][0]

In [16]:
v

Pretty

In [17]:
t.add_vibe(v)

dict_values([15201166620, 166620, 15201, 1.0])
<class 'dict_values'>
[<class 'int'>, <class 'int'>, <class 'int'>, <class 'float'>]

        INSERT OR IGNORE INTO track_vibes
            (track_vibe_id, vibe_id, track_id, value)
        VALUES
            (?, ?, ?, ?)


In [18]:
t.vibes()

[Energy 2, Pretty, Longing / Melancholy, Endings]

In [19]:
cursor.execute("SELECT * FROM track_vibes WHERE track_vibe_id=15201166620")

In [20]:
cursor.fetchall()

[{'track_vibe_id': 15201166620,
  'track_id': 15201,
  'vibe_id': 166620,
  'value': 1.0}]

In [21]:
t.track_id

15201